**This notebook contains a script that creates saves and tracks POE documents**

**Add local Library to Path**

In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

**Import Library**

In [4]:
import pandas as pd
import re
from local_library import create_folder
from local_library import import_worksheet
from local_library import export_worksheet
from local_library import signrequest_documents
from local_library import upload_to_gdrive

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-f1d5s7cpjrj54u2rqk1ufh9pevguqoap.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


**Import Data**

In [14]:
#import cohort data
cohort_df = import_worksheet("Cohort 2020 Data", "Cohort 2020 Data")

**Update Column Names**

In [15]:
column_names = {"Firstname":"learner_name", "Surname": "learner_surname",
                "Student Email":"learner_email", "Middle Name/s":"middle_name"
               }

cohort_df = cohort_df.rename(columns=column_names)

**Create Fullname**

In [16]:
cohort_df['middle_name'] = cohort_df['middle_name'].fillna("")
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['middle_name'].str.strip().str.lower()
cohort_df['learner_name'] = cohort_df['learner_name'].str.strip().str.title()
cohort_df['learner_fullname'] = cohort_df['learner_name'].str.strip().str.lower() + " " + cohort_df['learner_surname'].str.strip().str.lower()
cohort_df['learner_fullname'] = [re.sub(' +', ' ', fullname) for fullname in cohort_df['learner_fullname']]
cohort_df['learner_fullname'] = cohort_df['learner_fullname'].str.title()


**Subset Useful Columns**

In [17]:
column_names = ['Username', 'Campus', 'learner_email', 'learner_fullname']

cohort_df = cohort_df[cohort_df['Active / Not active/Early absorption'] == 'Active']
cohort_df = cohort_df[column_names]

**Create Folder ID Column**

In [18]:
cohort_df['folder_id'] = ''
len(cohort_df)

257

**Create Folders**

In [20]:
for index, row in cohort_df.iterrows():
    parent_id = create_folder("1W04tIGaWpqSgKsaP6xSjrUtP1OsVcaPX",row['learner_fullname'])
    cohort_df.loc[cohort_df['learner_fullname'] == row['learner_fullname'], 'folder_id'] = parent_id

**Add template IDS**

In [83]:
cohort_df['assessment_contract'] = ''
cohort_df['preassessment_meeting'] = ''
cohort_df['poe_instructions'] = 'feb67241-52f8-456c-917e-bdd0526619ee'
cohort_df['learner_contract'] = '900be1ba-0b96-4386-a904-fb67069f168e'
cohort_df['appeals_procedure'] = '589d226b-1c7e-441f-b99e-1d30a2d37761'
cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'assessment_contract'] = 'b48127ae-d278-467a-89b6-f6b3f010edb4'
cohort_df.loc[cohort_df['Campus'] == 'Cape Town', 'assessment_contract'] = '9a6b58fc-a6f6-4fba-8508-4ef859d98088'
cohort_df.loc[cohort_df['Campus'] == 'Johannesburg', 'preassessment_meeting'] = '2cec54b7-0ce3-4bd2-a9c5-29badf20d4f4'
cohort_df.loc[cohort_df['Campus'] == 'Cape Town', 'preassessment_meeting'] = '83797d10-f3d4-409d-aa39-93565ccd60fd'
cohort_df.loc[cohort_df['Campus'] == 'Durban', 'assessment_contract'] = '9a6b58fc-a6f6-4fba-8508-4ef859d98088'
cohort_df.loc[cohort_df['Campus'] == 'Durban', 'preassessment_meeting'] = '83797d10-f3d4-409d-aa39-93565ccd60fd'

**Save Database to Spreadsheet**

In [90]:
export_worksheet("POE Compilation","POE Documents", cohort_df)

**Find unique Template IDs and Emails**

In [85]:
def relevant_keys(target_columns):
    """
    Checks that all relevant keys are present.
    
    Args:
        target_columns: All columns of a dataframe.
        
    Returns:
        _: A boolean that indicates that all keys are relevant or not.
    """
    
    if 'template_id' not in target_columns:
        return False
    if 'email' not in target_columns:
        return False
    if 'signrequest_status' not in target_columns:
        return False
    if 'document_url' not in target_columns:
        return False
    return True


def can_upload(item):
    '''
    Uploads files to google drive.
    
    Args:
        item: a json objects from signrequests.
        
    Returns:
        _: a boolean indicating success or failure
    '''
    
    if relevant_keys(item.keys()) and item['signrequest_status'] in ["signed", "signed and downloaded"]:
        return True
    return False


def get_unique_values(target_df, columns):
    """
    Gets all the unique values in a(the) specified column(s).
    
    Args:
        target_df: A datframe containing the values in one or more columns.
        columns: A string or list of strings representing value containing columns.
        
    Returns:
        _: A list of unique values.
    """
    
    target_columns = list(filter(lambda column: True if column in columns else False ,target_df.columns))
    return pd.unique(target_df[target_columns].values.ravel('K'))


def get_documents_metadata(target_df):
    """
    Gets API metadata for all signrequests for a particular template.
    
    Args:
        source_df: A dataframe
        download_df: A dataframe.
        
    Returns:
        signrequests_list: A list of all relevant SignRequests.
    """
    
    #set variables
    page_number = 1
    
    #get unique template ids and emails
    emails = get_unique_values(target_df, "learner_email")
    template_ids = get_unique_values(target_df, ["preassessment_meeting"])
    
    #get pages
    while True:
        specific_filtered_results, next_page = signrequest_documents(emails, template_ids, page_number)
        for item in specific_filtered_results:
            if can_upload(item):
                if upload_to_gdrive(item['document_url'], 'preassessment_meeting', target_df[target_df['learner_email'] == item['email']].folder_id.values[0]):
                    target_df.loc[target_df['learner_email'] == item['email'], 'preassessment_meeting_status'] = 'Saved'
                else:
                    print(f"{item['email']} has failed to upload")
        if next_page:
            page_number += 1
        else:
            break

    return 0
        

In [86]:
get_documents_metadata(cohort_df)

2
257


**Save to POE tracking sheet**

In [92]:
export_worksheet("2020 SETA Submission Tracking Sheet","POE Documents Data Dump", cohort_df[['Username', 'Campus', 'poe_instructions_status', 'learner_contract_status', 'appeals_procedure_status', 'assessment_contract_status', 'preassessment_meeting_status']])


In [81]:
cohort_df['preassessment_meeting_status'] = ''

In [5]:
cohort_df = import_worksheet("POE Compilation","POE Documents")

In [11]:
cohort_df = cohort_df[cohort_df.isnull().any(axis=1)]

In [13]:
export_worksheet("POE Compilation","Follow", cohort_df[['Username', 'learner_email', 'poe_instructions_status', 'learner_contract_status', 'appeals_procedure_status', 'assessment_contract_status', 'preassessment_meeting_status']])
